In [1]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import urllib

import geocoder 
from geopy.geocoders import Nominatim

import folium 

import requests 
import json 
from pandas.io.json import json_normalize 

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

%matplotlib inline
print('Libraries imported.')

Libraries imported.


In this project data was scraped from [wikipedia page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) using BeautifulSoup library (for more information [link here](https://beautiful-soup-4.readthedocs.io/en/latest/#)).

In [2]:
page = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = BeautifulSoup(page)

In [3]:
table = soup.find_all('table')
df = pd.read_html(str(table))[0]
df.head()

Postal Code           Borough              Neighbourhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront

In [4]:
df = df.loc[(df['Borough']!='Not assigned') & (df['Neighbourhood']!='Not assigned')]
df.reset_index(drop=True,inplace=True)

In 'Neighbourhood' columns for one borough there are more then one neighbourhoods, so let's change DataFrame with only one neighbourhoods in row.    

In [5]:
lendf = df.shape[0]
for i in range(lendf):
    lst = df['Neighbourhood'][i].split(sep = ',')
    pc = df['Postal Code'][i]
    b = df['Borough'][i]
    
    for l in range(len(lst)):
        line = pd.DataFrame({'Postal Code': pc,'Borough': b,'Neighbourhood': lst[l]},index=[0])
        df = df.append(line,ignore_index=True)
        
df = df.loc[103:,:]
df.reset_index(drop=True,inplace=True)

In [6]:
df

Postal Code           Borough              Neighbourhood
0           M3A        North York                  Parkwoods
1           M4A        North York           Victoria Village
2           M5A  Downtown Toronto                Regent Park
3           M5A  Downtown Toronto               Harbourfront
4           M6A        North York             Lawrence Manor
..          ...               ...                        ...
212         M8Z         Etobicoke                  Mimico NW
213         M8Z         Etobicoke         The Queensway West
214         M8Z         Etobicoke             South of Bloor
215         M8Z         Etobicoke   Kingsway Park South West
216         M8Z         Etobicoke      Royal York South West

[217 rows x 3 columns]

Now, get the coordinates for every neighbourhood using Nominatim search engine with Forsquare agent.

In [7]:
ll = pd.DataFrame(columns=['Latitude','Longitude'])
for post in df['Neighbourhood'].values:
    geolocator = Nominatim(user_agent="foursquare_agent")
    
    g = geolocator.geocode('{}, Toronto, Canada'.format(post))
    #exeptions if response is Null object
    if (g is None):
        latitude = float('nan')
        longitude = float('nan')
    else:
        latitude = g.latitude
        longitude = g.longitude
    
    ll = ll.append({'Latitude': latitude,'Longitude': longitude},ignore_index=True)

ll.shape

(217, 2)

In [8]:
df = df.join(ll)
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)

In [9]:
df.head(10)

Postal Code           Borough           Neighbourhood   Latitude  Longitude
0         M3A        North York               Parkwoods  43.761124 -79.324059
1         M4A        North York        Victoria Village  43.732658 -79.311189
2         M5A  Downtown Toronto             Regent Park  43.660706 -79.360457
3         M5A  Downtown Toronto            Harbourfront  43.640080 -79.380150
4         M6A        North York          Lawrence Manor  43.722079 -79.437507
5         M6A        North York        Lawrence Heights  43.722778 -79.450933
6         M7A  Downtown Toronto            Queen's Park  43.659659 -79.390340
7         M9A         Etobicoke        Islington Avenue  43.620087 -79.512783
8         M9A         Etobicoke   Humber Valley Village  43.666472 -79.524314
9         M1B       Scarborough                 Malvern  43.809196 -79.221701